In [2]:
# =============================================================================
# Biblioteca
# =============================================================================
import time     # Facilita desenvolvimento envolvendo tempo, tempo de espera, etc
import pandas as pd
import glob # busca de arquivos em um diretório usando padrões de nome de arquivo.
import os
import shutil #Para mover um arquivo para outra pasta, você pode usar a função shutil.move() do módulo shutil.
from datetime import datetime
import sys
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import github

# pd.set_option('max_columns', None)
#pd.set_option('display.max_rows', None) #visualizar todas as colunas
# pd.set_option('max_columns', None)
# pd.set_option('max_columns') voltar ao normal; quantidade de colunas visualizadas
#Chrome
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import Select, WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC

ModuleNotFoundError: No module named 'github'

pip install selenium --proxy=http://array

In [ ]:
# =============================================================================
# Funções internas
# =============================================================================
sys.path.insert(1, '../wfuncoes py')
import keys as wk
import excel as we
import hana as wh
log = 1 #EXIBIR RESULTADOS

In [3]:
!git add main.ipynb

'git' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


### 1.EXTRAIR BASE DO HEMERA

Automação de acessar Hemera, baixar arquivo e salvar em pasta específica
- 1.01 Acessar o Hemera Coelba
- 1.02 Baixar Relatório de Situação de Status dos Medidores e mover para pasta raiz


In [ ]:
# # =============================================================================
# # 1.01 Login Hemera Coelba
# # =============================================================================
if log:
    print("=== Begin: 1 ===")
    print("=== Begin: 1.01 Acessar o Hemera Coelba ===")
wait = 0.2
link = "http://10.0.65.114:8080/hemera_coelba/loginHemera.jsp"

# Configurar opções do Edge
edge_options = Options()
edge_options.add_argument("--start-maximized")

# Iniciar o serviço do Edge WebDriver
service = Service()
nav = webdriver.Edge(service=service, options=edge_options)

# Navegar para uma página
nav.get(link)
time.sleep(wait)

# Inserir usuário
campo_usuario = WebDriverWait(nav, 15).until(EC.presence_of_element_located((By.XPATH, "//input[@name='username']")))
campo_usuario.send_keys(wk.user_rede_u)
time.sleep(wait)

# Inserir senha
campo_senha = WebDriverWait(nav, 5).until(EC.presence_of_element_located((By.XPATH, "//input[@name='password']")))
campo_senha.send_keys(wk.pass_rede)
time.sleep(wait)

# Selecionar domínio
campo_dominio =WebDriverWait(nav,5).until(EC.presence_of_element_located((By.XPATH,
                                     '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/form/div/div[5]/div/div/input[2]')))
campo_dominio.click()
time.sleep(wait)

# Selecionar domínio AMER
dominio_coelba = WebDriverWait(nav, 10).until(EC.presence_of_element_located((By.XPATH,
                                                                                         '/html/body/div[3]/div/div[1]')))
# /html/body/div[3]/div/div[1]
dominio_coelba.click()
time.sleep(wait)

#Clicar em LOGIN
botao_login = WebDriverWait(nav, 5).until(EC.presence_of_element_located((By.ID, 'divCenterButton')))
botao_login.click()

# Espera frame carregar
WebDriverWait(nav, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'iframe')))
# Pega o XPath do iframe da página inicial
iframe_pg_inicial = nav.find_element(By.TAG_NAME, 'iframe')
# Muda o foco para o iframe
nav.switch_to.frame(iframe_pg_inicial)

#Fechar pop-up de dicas
try:
    botao_fechar_pop_up = WebDriverWait(nav, 2).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[3]/div[4]/div')))
    botao_fechar_pop_up.click()
except:
    #             print("Não tem tela de dica")
        pass

time.sleep(wait)

if log:
    print("=== End: 1.01 ===")
    
# =============================================================================
# 1.02 Baixar Relatório de Situação de Status dos Medidores
# =============================================================================    
if log:
    print("=== Begin: 1.02 Baixar o Relatório de Situação de Status dos Medidores ===")

# Click Relatório Grupo B - menu
botao_grupo_B_superior = WebDriverWait(nav, 5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="ext-gen85"]/span')))
botao_grupo_B_superior.click()
time.sleep(wait)
# Click Relatório Situação de Status dos medidores
botao_rel_rscm = WebDriverWait(nav, 5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="mnu_relat"]/li[9]/a')))
botao_rel_rscm.click()
time.sleep(wait)
time.sleep(wait)
# Muda o foco para o iframe
nav.switch_to.default_content()
iframe_pg_inicial = nav.find_element(By.TAG_NAME, 'iframe')
nav.switch_to.frame(iframe_pg_inicial)
time.sleep(wait)
# click exportar
botao_exportar_csv = WebDriverWait(nav, 15).until(EC.presence_of_element_located((By.XPATH,'//*[@id="ext-gen34"]')))
botao_exportar_csv.click() 

# Aguardar finalizar download
pasta_padrao_downloads_edge = r"C:\Users\u461539\Downloads"
data_atual = datetime.now()
data_formatada = data_atual.strftime('%Y%m%d')
padrao_nome_arquivo = f"reportStatusCutMeter_{data_formatada}_*.csv"
#         print(padrao_nome_arquivo)
caminho_arquivo = os.path.join(pasta_padrao_downloads_edge, padrao_nome_arquivo)

if log:
    print("Download iniciado!")

while len(glob.glob(caminho_arquivo)) == 0:
    # Aguardar um tempo antes de verificar novamente
    time.sleep(2)

if log:
    print("Download concluído!")

# Apagar qualquer arquivo que esteja na pasta
pasta_root = r"C:\Users\u461539\OneDrive - IBERDROLA S.A\005.Scripts\SMC_BASE_HEMERA\input"
files = glob.glob(os.path.join(pasta_root, '*'))
for file in files:
    os.remove(file)
    
# pegar arquivo rscm.csv e mover para pasta_root:
time.sleep(2)
list_of_files = glob.glob(os.path.join(pasta_padrao_downloads_edge, "*.csv")) # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
novo_caminho = os.path.join(pasta_root, os.path.basename(latest_file))
shutil.move(latest_file, novo_caminho)
time.sleep(1)
if log:
    print("Novo arquivo movido para pasta!")

nav.quit()         # Fechar navegador

if log:
    print("=== End: 1.02 ===")

### 2.TRANSFORMADA DA BASE RSCM HEMERA

Importar arquivo CSV e realizar as transformadas do RSCM
- 2.01 Carregar dados
- 2.02 Delete columns e rows desnecessárias & Expurgos
- 2.03 Formatar colunas dt_ult_leitura, leitura_kwh GRN e LNG
- 2.04 Criar coluna net
- 2.05 Atualizar Rede a partir das net's em Hemera
    Função rede_input
    Check de integridade tb_rede
- 2.06 Expurgar medidores duplicados na mesma posição
- 2.07 Expurgar medidores duplicados em posição diferente
- 2.08 Expurgar medidores diferentes em mesma net_cs_posicao
- 2.09 Exportar rscm e rscm_expurgo
    Exportar para Hana

In [ ]:
# =============================================================================
# 2.01 Carregar dados
# =============================================================================
if log:
    print("=== Begin: 2 ===")
    print("=== Begin: 2.01 Carregar dados === ")

arq = we.read_csv("reportStatusCutMeter",log)
dt_ref = arq['dt_update']
rscm = arq['tabela']
rscm.columns =rscm.columns.str.lower()

if log:
    print(rscm.shape, "rscm")
    print("=== End: 2.01 ===")
    print('')
    
# =============================================================================
# 2.02 Delete columns e rows desnecessárias & Expurgos
# =============================================================================
if log:
    print("=== Begin: 2.02 Delete columns e rows desnecessárias ===")

rscm_expurgo = rscm.loc[rscm['regiao'] == 'Elektro']  # agrupamento Elektro
rscm_expurgo = rscm_expurgo.assign(tipo_erro=[f'regiao elektro' for i in range(len(rscm_expurgo))])
rscm = rscm.loc[rscm['regiao'] != 'Elektro'].copy()
if log:
    print(rscm_expurgo.shape, "rscm_expurgo: agrupamento Elektro")
rscm_expurgo = rscm_expurgo.loc[rscm_expurgo['regiao'] != 'Elektro'].copy()  # agrupamento Elektro

# rscm_expurgo: agrupamentos não inicia com G ou L
rscm_expurgo_new = rscm.loc[
    ~(rscm['agrupamento'].str.startswith('G') | rscm['agrupamento'].str.startswith('L'))]
rscm_expurgo_new = rscm_expurgo_new.assign(
    tipo_erro=[f'agrupamento não inicia com G ou L' for i in range(len(rscm_expurgo_new))])
rscm_expurgo = pd.concat([rscm_expurgo, rscm_expurgo_new])
rscm = rscm.loc[rscm['agrupamento'].str.startswith('G') | rscm['agrupamento'].str.startswith('L')].copy()
if log:
    print(rscm_expurgo_new.shape, "rscm_expurgo: agrupamentos não inicia com G ou L")
rscm_expurgo = rscm_expurgo.loc[
    rscm_expurgo['agrupamento'].str.startswith('G') | rscm_expurgo['agrupamento'].str.startswith('L')].copy()

# rscm_expurgo: medidores sem leitura
rscm_expurgo_new = rscm.loc[rscm['dt_ult_leitura'] == '']
rscm_expurgo_new = rscm_expurgo_new.assign(
    tipo_erro=[f'medidores sem leitura' for i in range(len(rscm_expurgo_new))])
rscm_expurgo = pd.concat([rscm_expurgo, rscm_expurgo_new])
rscm = rscm.loc[rscm['dt_ult_leitura'] != ''].copy()
if log:
    print(rscm_expurgo_new.shape, "rscm_expurgo: medidores sem leitura")

# rscm_expurgo: medidores maior que 10 digitos
rscm_expurgo_new = rscm.loc[rscm.em.apply(lambda x: len(x)) > 10]
rscm_expurgo_new = rscm_expurgo_new.assign(
    tipo_erro=[f'medidores maior que 10 digitos' for i in range(len(rscm_expurgo_new))])
rscm_expurgo = pd.concat([rscm_expurgo, rscm_expurgo_new])
rscm = rscm.loc[~(rscm.em.apply(lambda x: len(x)) > 10)].copy()
if log:
    print(rscm_expurgo_new.shape, "rscm_expurgo: medidores maior que 10 digitos")

# rscm_expurgo: medidores menor que 10 digitos
rscm_expurgo_new = rscm.loc[rscm.em.apply(lambda x: len(x)) < 10]
rscm_expurgo_new = rscm_expurgo_new.assign(
    tipo_erro=[f'medidores menor que 10 digitos' for i in range(len(rscm_expurgo_new))])
rscm_expurgo = pd.concat([rscm_expurgo, rscm_expurgo_new])
# display(rscm_expurgo_new)
rscm = rscm.loc[~(rscm.em.apply(lambda x: len(x)) < 10)].reset_index(drop=True).copy()
# display(rscm)
if log:
    print(rscm_expurgo_new.shape, "rscm_expurgo: medidores menor que 10 digitos")

# rscm_expurgo: medidores fora da faixa
rscm_expurgo_new = rscm[~rscm['em'].between('1000000000', '1299999999')]
rscm_expurgo_new = rscm_expurgo_new.assign(
    tipo_erro=[f'num patrimonio fora da faixa 1.000.000.000 e 1.299.999.999' for i in
               range(len(rscm_expurgo_new))])
rscm_expurgo = pd.concat([rscm_expurgo, rscm_expurgo_new])
rscm = rscm.loc[rscm['em'].between('1000000000', '1299999999')].copy()
if log:
    print(rscm_expurgo_new.shape, "rscm_expurgo: num patrimonio fora da faixa")

# exclui as colunas uc, regiao e tecnologia
rscm = rscm[['em', 'agrupamento', 'cs', 'posicao', 'dt_ult_leitura', 'leitura_kwh', 'status_rele']].copy()

if log:
    print(rscm_expurgo.shape, "rscm_expurgo")
    print(rscm.shape, "rscm")
    print("=== End: 2.02 ===")
    print('')
    
# =============================================================================
# 2.03 Formatar colunas dt_ult_leitura, leitura_kwh GRN e LNG,
# =============================================================================
if log:
    print("=== Begin: 2.03 Formatar colunas dt_ult_leitura, leitura_kwh GRN e LNG ===")

# Convertendo a coluna "leitura_kwh" para um tipo numérico adequado
rscm.loc[:, 'dt_ult_leitura'] = pd.to_datetime(rscm['dt_ult_leitura'], format='%d/%m/%Y')
rscm['leitura_kwh'] = rscm['leitura_kwh'].str.replace('[\.,-]', '', regex=True).str[:-4]
rscm['leitura_kwh'] = rscm.apply(
    lambda row: 0 if row['agrupamento'].startswith('G') and len(row['leitura_kwh']) < 1 else \
        (0 if row['agrupamento'].startswith('L') and len(str(row['leitura_kwh'])) <= 3 else \
             row['leitura_kwh'][:-3] if row['agrupamento'].startswith('L') and row['leitura_kwh'].isdigit() else \
                 row['leitura_kwh']), axis=1).astype(int)

if log:
    print(rscm.info())
    print("=== End: 2.03 ===")
    print('')

# =============================================================================
# 2.04 Criar coluna net
# =============================================================================
if log:
    print("=== Begin: 2.04 Criar coluna net ===")

try:
    rscm['net'] = rscm['agrupamento'].str[4:8].astype(int)
except:
    print("Erro 2.04: Tem agrupamento em formatação indevida.")

rscm = rscm[['net', 'cs', 'em', 'posicao', 'dt_ult_leitura', 'leitura_kwh', 'status_rele',
               'agrupamento']].copy().sort_values(by=['net', 'cs', 'posicao', 'em'])

if log:
    print(rscm.keys())
    print("===End: 2.04 ===")
    print('')

# =============================================================================
# 2.05 Atualização das redes
# =============================================================================
#Não tem atualização da base de redes no HANA
pass

# =============================================================================
# 2.06 Expurgar medidores duplicados na mesma posição
#   Expurgar medidores duplicados mesma posição-> saída rscm_expurgo
#   Retirar de rscm as linhas duplicadas
# =============================================================================
if log:
    print("=== Begin: 2.06 Expurgar medidor duplicado em net_cs_pos igual ===")

rscm = rscm[['net', 'cs', 'posicao', 'em', 'dt_ult_leitura', 'leitura_kwh', 'status_rele',
               'agrupamento']].copy().sort_values(by=['net', 'cs', 'posicao', 'em'])
duplicatas = rscm.duplicated(subset=['net', 'cs', 'em', 'posicao', 'dt_ult_leitura', 'leitura_kwh'],
                              keep='first')

rscm_expurgo_new = rscm[duplicatas].drop_duplicates()
try:
    rscm_expurgo_new.loc[:, 'tipo_erro'] = 'medidor duplicado em net_cs_pos igual'
except Exception as e:
    print(f"Não encontrado 'medidor duplicado em net_cs_pos igual': {e}")

if log:
    print(rscm_expurgo_new.shape, 'rscm_expurgo medidor duplicado em net_cs_pos igual')
rscm_expurgo = pd.concat([rscm_expurgo, rscm_expurgo_new], ignore_index=True)
if log:
    print(rscm_expurgo.shape, 'rscm_expurgo final')
# tirar de rscm as linhas duplicadas
rscm = rscm.drop_duplicates(subset=['net', 'cs', 'em', 'posicao', 'dt_ult_leitura', 'leitura_kwh']).copy()
if log:
    print(rscm.shape, 'rscm')
    print(rscm_expurgo.shape, 'rscm_expurgo')
    print("===End: 2.06 ===")
    print('')

# =============================================================================
# 2.07 Expurgar medidores duplicados em posição diferente
#   Expurgar medidores duplicados em posição diferente -> saída rscm_expurgo
#   Retirar de rscm as linhas duplicadas
# =============================================================================
if log:
    print("=== Begin: 2.07 Expurgar medidor duplicado em net_cs_pos diferente ===")

rscm = rscm.copy().sort_values(by=['em', 'dt_ult_leitura', 'net', 'cs', 'posicao'],
                                 ascending=[True, False, True, True, True]).reset_index(drop=True)
duplicatas = rscm.duplicated(subset=['em'], keep=False)
rscm_duplicatas = rscm[duplicatas].reset_index(drop=True)
try:
    rscm_duplicatas.loc[:, 'tipo_erro'] = 'medidor duplicado em net_cs_pos diferente'
except Exception as e:
    print(f"Não encontrado 'medidor duplicado em net_cs_pos diferente': {e}")

if log:
    print(rscm_duplicatas.shape, 'rscm_expurgo medidor duplicado em net_cs_pos diferente')
rscm_expurgo = pd.concat([rscm_expurgo, rscm_duplicatas], ignore_index=True)
if log:
    print(rscm_expurgo.shape, 'rscm_expurgo final')
# tirar de rscm as linhas duplicadas
rscm = rscm.drop_duplicates(subset=['em']).copy()

if log:
    print(rscm.shape, 'rscm')
    print("===End: 2.07 ===")
    print('')

# =============================================================================
# 2.08 Expurgar medidor diferente na mesma net_cs_pos
#   Selecionar medidores diferentes em mesma net_cs_posicao -> saída rscm_expurgo
#   Manter rscm
# =============================================================================
if log:
    print("=== Begin: 2.08 Expurgar medidores diferentes na mesma net_cs_pos ===")

rscm = rscm.copy().sort_values(by=['net', 'cs', 'posicao', 'dt_ult_leitura'],
                                 ascending=[True, True, True, False]).reset_index(drop=True)
duplicatas = rscm.duplicated(subset=['net', 'cs', 'posicao'], keep=False)
rscm_duplicatas = rscm[duplicatas].reset_index(drop=True)
rscm_duplicatas.loc[:, 'tipo_erro'] = 'medidores diferentes na mesma net_cs_pos'
if log:
    print(rscm_duplicatas.shape, 'rscm_expurgo medidores diferentes na mesma net_cs_pos')
rscm_expurgo = pd.concat([rscm_expurgo, rscm_duplicatas], ignore_index=True)

if log:
    print(rscm.shape, 'rscm')
    print(rscm_expurgo.shape, 'rscm_expurgo')
    print("===End: 2.08 ===", end='\n')
    print('')

# =============================================================================
# 2.09 Exportar rscm e rscm_expurgo e dt_update
#   Adicionar a datahora que o arquivo foi baixado
#   Exportar para Excel
#   Exportar para Hana
# =============================================================================
if log:
    print("=== Begin: 2.09 Adicionar Datahora e exportar ao  Excel e HANA ===")

rscm['dt_ref'] = dt_ref

# Exportar Excel
rscm.to_csv(r'.\output\\rscm.csv', sep=';', index=False)
rscm_expurgo.to_excel(".\output\\rscm_expurgo.xlsx", sheet_name='Plan1',index=False)
if log:
    print('exportado Excel rscm.csv e rscm_expurgo.xlsx')

# Exportar Hana
#wh.hana_insert(rscm, 'SMC_HEMERA', log)

if log:
    print("===End: 2.09 ===", end='\n')
    print('')
